In [18]:
import numpy as np
from helper import *

In [43]:
# Parameters
nx, ny = 101, 101
xmin = 0
xmax = 1.0
ymin = -0.5
ymax = 0.5

Lx = xmax - xmin
Ly = ymax - ymin
dx = Lx / (nx - 1)
dy = Ly / (ny - 1)

# Meshing x, y-directions
x = np.linspace(xmin, xmax, num=nx)
y = np.linspace(ymin, ymax, num=ny)

X, Y = np.meshgrid(x, y) #use this to convert x,y into 2d arrays since linspace 
                        # is a 1d operator, meshgrid fills in grid of size nx, ny

# the source term, b which is a 2d array now
b = -2 * (np.pi/2)**2 * np.sin(np.pi * X / Lx) * np.cos(np.pi * Y / Ly)

# initial conditions :: solution p initially filled with 0 
p0 = np.zeros((ny, nx))

# Analytical Solution
p_exact = poisson_solution(x, y, Lx, Ly)

### Steepest Descent Method
##### 1. Calculate residual, $r^k$, which also serves as direction vector $d^k$
##### 2. Calculate step size $\alpha$
##### 3. Update $p^{k+1}$ = $p^k$ + $\alpha$$d^k$

In [44]:
# Poisson 2d Steepest Descent Function
def poisson_2d_steepest_descent(p0, b, dx, dy, maxiter=20000, rtol=1e-6):
    def A(p):
        return (-4 * p[1:-1, 1:-1] + 
                     p[1:-1, :-2] + 
                    p[1:-1, 2:] + 
                    p[:-2, 1:-1] + 
                    p[2:, 1:-1] ) / dx**2
    p = p0.copy()
    rk = np.zeros_like(p)    # initial residual
    Ar = np.zeros_like(p) # stores matrix-vector multiplication values
    conv = [] #empty set to store convergence history
    diff = rtol + 1
    ite = 0
    while diff > rtol and ite < maxiter:
        pk = p.copy()
        # Time to compute residuak, r^k
        rk[1:-1, 1:-1] = b[1:-1, 1:-1] - A(pk)
        
        # Compute laplacian of residual
        Ar[1:-1, 1:-1] = A(rk)
        
        # Compute step size, alpha
        alpha = np.sum(rk * rk) / np.sum(rk * Ar)
        
        # Update solution
        p = pk + alpha * rk
        
        #Dirichlet BC enforced automatically (why?)
        # L2-Norm is calculated
        diff = l2_norm(p,pk)
        conv.append(diff)
        ite += 1
    return p, ite, conv

In [45]:
# Time to compute solution using this method
p, ites, conv_sd = poisson_2d_steepest_descent(p0, b, dx, dy,
                                               maxiter=20000, 
                                               rtol=1e-10)

print('Method of Steepest Descent: {} iterations'.format(ites) + 
     ' to reach a relative difference of {}'.format(conv_sd[-1]))


Method of Steepest Descent: 2 iterations to reach a relative difference of 1.3307695446303778e-16


In [46]:
# Compute Relative L2 Norm of Error
err = l2_norm(p, p_exact)
print('The relative l2_norm of the error is: {}'.format(err))

The relative l2_norm of the error is: 0.7499794373094477


## Conjugate Gradients
##### Before: Calculate ${\bf d}^{0} = {\bf r}^{0}$ (just once)
##### 1. Calculate $\alpha$ = $\frac{r^{k}\cdot r^{k}}{Ad^{k} \cdot d^{k}}$
##### 2. Update $p^{k+1}$
##### 3. Calculate $r^{k+1} = r^{k} - \alpha A d^{k} $
##### 4. Calculate $\beta$ = $\frac{r^{k+1} \cdot r^{k+1}}{r^{k} \cdot r^{k}}$
##### 5. Calculate $d^{k+1} = r^{k+1} + \beta d^{k} $
##### 6. Repeat

In [49]:
def poisson_2d_conjugate_gradient(p0, b, dx, dy, maxiter=20000, rtol=1e-6):
    def A(p):
        return (-4.0 * p[1:-1, 1:-1] + 
                       p[1:-1, :-2] + p[1:-1, 2:] +
                        p[:-2, 1:-1] + p[2:, 1:-1] ) / dx**2
    p = p0.copy()
    r = np.zeros_like(p)
    Ad = np.zeros_like(p)
    conv = []
    diff = rtol + 1
    ite = 0
    r[1:-1, 1:-1] = b[1:-1, 1:-1] - A(p)
    d = r.copy()
    while diff > rtol and ite < maxiter:
        pk = p.copy()
        rk = r.copy()
        # Laplace of Search Direction
        Ad[1:-1, 1:-1] = A(d)
        
        # Compute Step Size
        alpha = np.sum(r*r) / np.sum(d*Ad)
        
        # Update Solution
        p = pk + alpha * d
        
        # Update Residual
        r = rk - alpha * Ad
        
        # Update search direction
        beta = np.sum(r * r) / np.sum(rk * rk)
        d = r + beta * d
        
        # Direchlet BC automatically enforced (i.e. zero at boundary from np.zeros)
        # Compute relative L2 Norm
        diff = l2_norm(p, pk)
        conv.append(diff)
        ite += 1
    return p, ite, conv

In [50]:
# Compute Solution using Conugate Gradients
p, ites, conv_cg = poisson_2d_conjugate_gradient(p0, b, dx, dy,
                                                maxiter=20000, rtol=1e-10)
print('Method of Conj. Gradients: {} iterations '.format(ites) + 
     'to reach relative difference of {}'.format(conv_cg[-1]))

Method of Conj. Gradients: 2 iterations to reach relative difference of 1.2982770796281907e-16


In [51]:
# Compute L2_Norm of Error
l2_norm(p, p_exact)

0.7499794373094477

In [52]:
# Solution computed via Jacobi Relaxation
p, ites, conv_jacobi = poisson_2d_jacobi(p0, b, dx, dy, 
                                        maxiter=40000,
                                        rtol=1e-10)
print('For Jacobi Relaxation: {} iterations '.format(ites) + 
     'to reach relative difference of {}'.format(conv_jacobi[-1]))

For Jacobi Relaxation: 31227 iterations to reach relative difference of 9.997923503623598e-11


## More Difficult Poisson Problems

In [55]:
# Modify Source Term
b = (np.sin(np.pi * X / Lx) * 
     np.cos(np.pi * Y / Ly) + 
    np.sin(6.0 * np.pi * X / Lx) * 
     np.cos(6.0 * np.pi * Y / Ly))

In [56]:
maxiter, rtol = 40000, 1e-10
p, ites, conv = poisson_2d_jacobi(p0, b, dx, dy, maxiter=maxiter, rtol=rtol)

print('Jacobi Relaxation: {} iterations'.format(ites))

p,ites,conv = poisson_2d_steepest_descent(p0, b, dx, dy, maxiter=maxiter,
                                         rtol=rtol)
print('Steepest Descent: {} iterations'.format(ites))

p,ites,conv = poisson_2d_conjugate_gradient(p0, b, dx, dy, maxiter=maxiter, 
                                            rtol=rtol)

print('Conjugate Gradients: {} iterations'.format(ites))

Jacobi Relaxation: 31226 iterations
Steepest Descent: 31591 iterations
Conjugate Gradients: 72 iterations
